# Black-Litterman Allocation Model
This notebook mirrors the original R workflow while translating every step into Python.

## Environment setup
Import numerical and optimization libraries required to reproduce the asset allocation exercise.

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
from IPython.display import display

np.set_printoptions(precision=6, suppress=True)

## Baseline market inputs
Define volatilities, benchmark weights, and CAPM-implied returns. These inputs let us infer the equilibrium returns that are consistent with the starting portfolio.

In [2]:
assets = np.array(['A', 'B', 'C', 'D'])
vol = np.array([0.15, 0.20, 0.25, 0.30])
w0 = np.array([0.4, 0.3, 0.2, 0.1])
mu_capm = np.array([0.05, 0.06, 0.08, 0.06])

correlation = np.array([
    [1.0, 0.1, 0.4, 0.5],
    [0.1, 1.0, 0.7, 0.4],
    [0.4, 0.7, 1.0, 0.8],
    [0.5, 0.4, 0.8, 1.0]
])

# Convert volatilities and correlation into a covariance matrix
diag_sd = np.diag(vol)
Sigma = diag_sd @ correlation @ diag_sd
Sigma_df = pd.DataFrame(Sigma, index=assets, columns=assets)

# Portfolio statistics implied by the benchmark weights
sd_p = float(np.sqrt(w0 @ Sigma @ w0))
rp = float(mu_capm @ w0)
SR_target = 0.25 #Sharpe Ratio 
risk_aversion = SR_target / sd_p
rf = 0.03 # Risk-free rate

mu_implied = rf + risk_aversion * (Sigma @ w0)
mu_implied_alt = rf + SR_target * (Sigma @ w0) / sd_p
risk_premium = mu_implied - rf
risk_premium_alt = SR_target * (Sigma @ w0) / sd_p

summary_table = pd.DataFrame({
    'Asset': assets,
    'Volatility': vol,
    'CAPM Return': mu_capm,
    'Implied Return': mu_implied,
    'Risk Premium': risk_premium
})

print('Portfolio return (CAPM-weighted):', round(rp, 6))
print('Portfolio volatility:', round(sd_p, 6))
print('Risk aversion coefficient:', round(risk_aversion, 6))
print('Implied returns consistency check:', np.allclose(mu_implied, mu_implied_alt))
print('Risk premium consistency check:', np.allclose(risk_premium, risk_premium_alt))
summary_table

Portfolio return (CAPM-weighted): 0.06
Portfolio volatility: 0.153493
Risk aversion coefficient: 1.628742
Implied returns consistency check: True
Risk premium consistency check: True


,Asset,Volatility,CAPM Return,Implied Return,Risk Premium
0,A,0.15,0.05,0.054675,0.024675
1,B,0.20,0.06,0.066810,0.036810
2,C,0.25,0.08,0.087006,0.057006
3,D,0.30,0.06,0.090589,0.060589


## Incorporating portfolio manager views
Encode the relative value views and compute the posterior (Black-Litterman) expected returns.

In [7]:
def markowitz_long_only(mu, Sigma, risk_aversion, rf, lambda_tradeoff=0.5):
    
    n = len(mu)
    w = cp.Variable(n)
    objective = cp.Minimize(lambda_tradeoff * cp.quad_form(w, Sigma) - (1 / risk_aversion) * (mu @ w - rf))
    constraints = [w >= 0, cp.sum(w) == 1]
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS)

    if problem.status not in (cp.OPTIMAL, cp.OPTIMAL_INACCURATE):
        raise ValueError(f'Optimization failed with status {problem.status}')

    return np.asarray(w.value).reshape(-1)

In [8]:
# Scenario 1: Asset A return will be 4% and Asset B will underperform Asset C by 1%%
# Confidence level: 10% for the first view and 5% for the second view
# View matrix P, view vector Q, and view uncertainty matrix Omega
P = np.array([
    [1, 0, 0, 0],
    [0, 1, -1, 0]
])
Q = np.array([0.04, -0.01])
omega = np.diag([0.2**2, 0.2**2])# Confidence level: 10% for the first view and 5% for the second view

tau = 0.1
rho = tau * Sigma

adjustment = rho @ P.T @ np.linalg.inv(P @ rho @ P.T + omega) @ (Q - P @ mu_implied)
mu_posterior = mu_implied + adjustment

posterior_table = pd.DataFrame({
    'Asset': assets,
    'Implied Return': mu_implied,
    'Posterior Return': mu_posterior,
    'View Adjustment': adjustment
})

print('Return shift from views:')
display(pd.Series(mu_posterior - mu_implied, index=assets))
posterior_table

Return shift from views:


A   -0.001039
B    0.000011
C   -0.001134
D   -0.001582
dtype: float64

,Asset,Implied Return,Posterior Return,View Adjustment
0,A,0.054675,0.053637,-0.001039
1,B,0.066810,0.066820,0.000011
2,C,0.087006,0.085872,-0.001134
3,D,0.090589,0.089007,-0.001582


## Optimizing the Black-Litterman portfolio
Solve the long-only Markowitz problem using the posterior returns and evaluate the tracking error versus the benchmark allocation.

In [9]:


w_bl = markowitz_long_only(mu_posterior, Sigma, risk_aversion, rf)
weights_comparison = pd.DataFrame({
    'Asset': assets,
    'Benchmark Weight': w0,
    'Black-Litterman Weight': w_bl,
    'Difference': w_bl - w0
})

tracking_error_vol = float(np.sqrt((w_bl - w0) @ Sigma @ (w_bl - w0)))

print('Tracking error volatility (%):', round(tracking_error_vol * 100, 4))
weights_comparison.round(4)

Tracking error volatility (%): 0.4097


,Asset,Benchmark Weight,Black-Litterman Weight,Difference
0,A,0.4,0.3943,-0.0057
1,B,0.3,0.3243,0.0243
2,C,0.2,0.1818,-0.0182
3,D,0.1,0.0997,-0.0003
